In [4]:
# Importar las librerías necesarias

from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt


In [7]:
def bb84_protocol(num_bits=50, eavesdrop=False):
    # Alice genera bits aleatorios y bases (Z o X)
    alice_bits = np.random.randint(2, size=num_bits)
    alice_bases = np.random.randint(2, size=num_bits)  # 0 = Z, 1 = X

    # Bob elige bases aleatorias
    bob_bases = np.random.randint(2, size=num_bits)

    # Alice codifica sus bits
    simulator = Aer.get_backend('qasm_simulator')
    qubits = []
    for bit, base in zip(alice_bits, alice_bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)
        if base == 1:  # Base X: aplicamos Hadamard
            qc.h(0)
        qubits.append(qc)

    # Eve intenta espiar (ataque de medida-reenvío)
    eve_bits = []
    if eavesdrop:
        eve_bases = np.random.randint(2, size=num_bits)
        for i in range(num_bits):
            qc = qubits[i].copy()
            if eve_bases[i] == 1:
                qc.h(0)
            qc.measure(0, 0)
            result = simulator.run(qc, shots=1).result()
            eve_bit = int(list(result.get_counts())[0], 2)
            eve_bits.append(eve_bit)

    # Bob mide los qubits recibidos
    bob_bits = []
    for i in range(num_bits):
        qc = qubits[i].copy()
        if bob_bases[i] == 1:  # Base X: aplicamos Hadamard antes de medir
            qc.h(0)
        qc.measure(0, 0)
        result = simulator.run(qc, shots=1).result()
        bob_bit = int(list(result.get_counts())[0], 2)
        bob_bits.append(bob_bit)

    # Comparan bases y extraen clave compartida
    key = []
    for i in range(num_bits):
        if alice_bases[i] == bob_bases[i]:
            key.append(bob_bits[i])

    print("Clave compartida:", key[:10], "...")
    
    # Si Eve espía, calculamos la tasa de error aproximada
    if eavesdrop:
        sample_size = min(10, len(key))
        sampled_indices = np.random.choice(range(len(key)), size=sample_size, replace=False)
        alice_sample = [key[i] for i in sampled_indices]
        bob_sample = [key[i] for i in sampled_indices]
        error_rate = sum(a != b for a, b in zip(alice_sample, bob_sample)) / sample_size
        print(f"Tasa de error detectada (indica espionaje): {error_rate * 100:.1f}%")
    
    return key
def guardar_clave(clave, archivo='clave.txt'):
    """
    Guarda una lista de bits (clave) en un archivo de texto.
    
    Parámetros:
        clave (list): Lista de enteros (0 o 1)
        archivo (str): Nombre del archivo donde se guardará la clave
    """
    with open(archivo, 'w') as f:
        f.write(''.join(str(bit) for bit in clave))
    print(f"✅ Clave guardada en '{archivo}'")

def leer_clave(archivo='clave.txt'):
    """
    Lee una clave desde un archivo de texto.
    
    Parámetros:
        archivo (str): Nombre del archivo desde el cual se leerá la clave
    
    Retorna:
        list: Lista de enteros (0 o 1)
    """
    with open(archivo, 'r') as f:
        contenido = f.read().strip()
    
    clave = [int(c) for c in contenido if c in ('0', '1')]
    print(f"✅ Clave leída desde '{archivo}': {clave[:10]}... ({len(clave)} bits)")
    return clave

In [6]:
key = bb84_protocol(eavesdrop=False)
print(key)
guardar_clave(key, 'clave.txt')

Clave compartida: [0, 1, 1, 0, 0, 1, 0, 1, 1, 0] ...
[0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0]
✅ Clave guardada en 'clave.txt'
